In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep

### initialised Nominatim 
geolocator = Nominatim(user_agent="property 4", timeout=10)

data = pd.read_csv('../data/curated/property/property 4.csv')


def geocode_address(address, retries=3):
    for attempt in range(retries):
        try:
            location = geolocator.geocode(address)
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except Exception as e:
            print(f"Error geocoding {address}: {e}, retrying ({attempt+1}/{retries})...")
            sleep(2)  ### waiting time
    return None, None

### divided the data into batches
def batch_geocode(data, batch_size=100, sleep_time=1):
    latitudes = []
    longitudes = []
    

    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        print(f"processing {i // batch_size + 1} th...")
        
        for address in batch['address']:
            lat, lon = geocode_address(address)
            latitudes.append(lat)
            longitudes.append(lon)
            sleep(sleep_time) 
        
        print(f" {i // batch_size + 1} th data finished。")
    
    return latitudes, longitudes

### batch size
batch_size = 100 
latitudes, longitudes = batch_geocode(data, batch_size=batch_size, sleep_time=2)

data['latitude'] = latitudes
data['longitude'] = longitudes

# save the fisrt convert data
csv_output_path = '../data/curated/property/property_4_lat_lon.csv'
data.to_csv(csv_output_path, index=False)

print(f"all address saved into {csv_output_path}")


In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
import re
from time import sleep


### this part is continue to deal with the pervious one
### first converting may remaining some properties cannot convert
### this is used to remove the unit number to convert the latitude and longitude

geolocator = Nominatim(user_agent="property 4", timeout=10)
data = pd.read_csv('../data/curated/property/property_4_lat_lon.csv')


def geocode_address(address, retries=3):
    for attempt in range(retries):
        try:
            location = geolocator.geocode(address)
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except Exception as e:
            print(f"Error geocoding {address}: {e}, retrying ({attempt+1}/{retries})...")
            sleep(2)  
    return None, None

### only remain the words in the address
def remove_numbers(address):
    return re.sub(r'[^a-zA-Z\s]', '', address).strip()

### get the latitude is empty ones
no_lat_lng = data[data['latitude'].isna() | data['longitude'].isna()].copy()

### using function
no_lat_lng['cleaned_address'] = no_lat_lng['address'].apply(remove_numbers)

### re get the latitude
no_lat_lng['latitude'], no_lat_lng['longitude'] = zip(*no_lat_lng['cleaned_address'].apply(geocode_address))

### save them
data.update(no_lat_lng[['latitude', 'longitude']])
csv_output_path = '../data/curated/property/property_4_lat_lon.csv'
data.to_csv(csv_output_path, index=False)

print(f"all address saved in {csv_output_path}")

In [ ]:
import pandas as pd

### read again
file_path = '../data/curated/property/property_4_lat_lon.csv'  
data = pd.read_csv(file_path)

### clean the data and save
cleaned_data = data.dropna(subset=['latitude', 'longitude'])
output_file_path = '../data/curated/property/property_4_lat_lon.csv' 
cleaned_data.to_csv(output_file_path, index=False)
